# Offline-aware Knowledge Tracing (OKT)

This notebook will show you how to train and use the OKT.
First, we will show how to get the data (here we use assistment-2017 as the dataset).
Then we will show how to train a LPKT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [OKT.py](OKT.ipynb)


## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
import numpy as np
from load_data import DATA

dataset_name = 'assist2017'

# dataset detail
n_at = 1326
n_it = 2873
n_exercise = 3162
n_question = 102
seq_len = 500
dataset_path = 'anonymized_full_release_competition_dataset'
has_at = True

# hyperparameters
d_q, d_e = 32, 128
d_p, d_a = 128, 128
d_at = 50
d_h = 128

# train detail
dropout = 0.3
batch_size = 64
lr = 3e-3
lr_decay_step = 10
lr_decay_rate = 0.5
epoch = 2

data_path = './data/' + dataset_path
dat = DATA(seqlen=seq_len, separate_char=',', has_at=has_at)
train_data = dat.load_data(data_path + '/train0.txt')
valid_data = dat.load_data(data_path + '/valid0.txt')
test_data = dat.load_data(data_path + '/test.txt')
model_file_path = 'okt-' + dataset_name + '.params'

## Training and Persistence

In [2]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [3]:
from EduKTM import OKT

model = OKT(n_at, n_it, n_exercise, n_question, d_e, d_q, d_a, d_at, d_p, d_h,
            batch_size=batch_size, dropout=dropout)
best_train_auc, best_valid_auc = model.train(train_data, valid_data,
                                            epoch=epoch, lr=lr, lr_decay_step=lr_decay_step,
                                            lr_decay_rate=lr_decay_rate,
                                            filepath=model_file_path)

Testing:   0%|          | 0/7 [00:00<?, ?it/s]

[Epoch 0] LogisticLoss: 0.623074


Testing: 100%|██████████| 7/7 [00:02<00:00,  2.40it/s]
INFO:root:save parameters to okt-assist2017.params
Training:   0%|          | 0/28 [00:00<?, ?it/s]

[Epoch 0] r2: 0.127399, auc: 0.705811, accuracy: 0.680212


Testing:   0%|          | 0/7 [00:00<?, ?it/s]

[Epoch 1] LogisticLoss: 0.575053


Testing: 100%|██████████| 7/7 [00:02<00:00,  2.47it/s]
INFO:root:save parameters to okt-assist2017.params


[Epoch 1] r2: 0.181005, auc: 0.748027, accuracy: 0.700344


## Loading and Testing

In [4]:
model.load(model_file_path)
_, test_rmse, test_r2, test_auc, test_accuracy = model.eval(test_data)
print("rmse: %.6f, r2: %.6f, auc: %.6f, accuracy: %.6f" % (test_rmse, test_r2, test_auc, test_accuracy))

INFO:root:load parameters from okt-assist2017.params
Testing: 100%|██████████| 9/9 [00:03<00:00,  2.56it/s]

rmse: 0.437290, r2: 0.184441, auc: 0.752023, accuracy: 0.709414
